# 서울시 코로나19 확진자 현황

In [1]:
import numpy as np
import pandas as pd

# An importlib package installable from any python version
import importlib
import modules.fs as fs

In [2]:
covid19 = fs.read_csv('./data/data.seoul.go.kr/corona19status/20201201/서울시 코로나19 확진자 현황.csv', encoding='euc-kr')

covid19['날짜'] = '2020.' + covid19['확진일'].replace('\.$', '', regex=True)
covid19['날짜'] = pd.to_datetime(covid19['날짜'], format='%Y.%m.%d')
covid19['년'] = covid19['날짜'].dt.year
covid19['월'] = covid19['날짜'].dt.month
covid19['일'] = covid19['날짜'].dt.day

covid19.head()

,연번,확진일,환자번호,국적,환자정보,지역,여행력,접촉력,조치사항,상태,이동경로,등록일,수정일,노출여부,날짜,년,월,일
0,8966,11.30.,NaN,NaN,NaN,강북구,NaN,감염경로 조사중,NaN,NaN,NaN,2020-12-01 15:37:10,2020-12-01 15:37:10,Y,2020-11-30,2020,11,30
1,8965,11.30.,NaN,NaN,NaN,타시도,NaN,기타 확진자 접촉,NaN,NaN,NaN,2020-12-01 15:37:10,2020-12-01 15:37:10,Y,2020-11-30,2020,11,30
2,8964,11.30.,NaN,NaN,NaN,타시도,NaN,감염경로 조사중,NaN,NaN,NaN,2020-12-01 15:37:10,2020-12-01 15:37:10,Y,2020-11-30,2020,11,30
3,8963,11.30.,NaN,NaN,NaN,영등포구,NaN,서초구 사우나 관련,NaN,NaN,NaN,2020-12-01 15:37:10,2020-12-01 15:37:10,Y,2020-11-30,2020,11,30
4,8962,11.30.,NaN,NaN,NaN,영등포구,NaN,감염경로 조사중,NaN,NaN,NaN,2020-12-01 15:37:10,2020-12-01 15:37:10,Y,2020-11-30,2020,11,30


## 서울시 전체 현황

In [3]:
# 상태 분석
si_status = covid19.shape[0]
si_discharged = covid19[covid19['상태'] == '퇴원'].shape[0]
si_dead = covid19[covid19['상태'] == '사망'].shape[0]
si_hospitalized = si_status - si_discharged - si_dead

df = pd.DataFrame({
    '퇴원': [si_discharged], 
    '사망': [si_dead], 
    '입원': [si_hospitalized]
})

fs.to_csv(df, './data/si_status.csv')

In [4]:
# 여행력 분석
si_notravel = covid19['여행력'].isna().sum()
si_traveled = covid19['여행력'].notna().sum()

df = pd.DataFrame({
    '집': [si_notravel], 
    '여행': [si_traveled]
})

fs.to_csv(df, './data/si_travel.csv')

In [5]:
# 확진일 일별 분석
si_daily_discharged = covid19[covid19['상태'] == '퇴원'].groupby('날짜').size()
si_daily_dead = covid19[covid19['상태'] == '사망'].groupby('날짜').size()
si_daily_hospitalized = covid19[covid19['상태'].isna()].groupby('날짜').size()

df = pd.DataFrame({
    '퇴원': si_daily_discharged, 
    '사망': si_daily_dead,
    '입원': si_daily_hospitalized
})

fs.to_csv(df, './data/si_daily.csv', index=True)

In [6]:
# 확진일 월별 분석
si_monthly_discharged = covid19[covid19['상태'] == '퇴원'].groupby(pd.Grouper(key='날짜', freq='1M')).size()
si_monthly_dead = covid19[covid19['상태'] == '사망'].groupby(pd.Grouper(key='날짜', freq='1M')).size()
si_monthly_hospitalized = covid19[covid19['상태'].isna()].groupby(pd.Grouper(key='날짜', freq='1M')).size()

df = pd.DataFrame({
    '퇴원': si_monthly_discharged, 
    '사망': si_monthly_dead,
    '입원': si_monthly_hospitalized
})

fs.to_csv(df, './data/si_monthly.csv', index=True)

## 지역구 분석

In [7]:
# 상태 분석
gu_status = covid19.groupby('지역').size()
gu_discharged = covid19[covid19['상태'] == '퇴원'].groupby('지역').size()
gu_dead = covid19[covid19['상태'] == '사망'].groupby('지역').size()
gu_hospitalized = gu_status - gu_discharged - gu_dead

df = pd.DataFrame({
    '퇴원': gu_discharged, 
    '사망': gu_dead,
    '입원': gu_hospitalized
})

fs.to_csv(df, './data/gu_status.csv', index=True)

In [8]:
# 여행력 분석
gu_notravel = covid19[covid19['여행력'].isna()].groupby('지역').size()
gu_traveled = covid19[covid19['여행력'].notna()].groupby('지역').size()

df = pd.DataFrame({
    '집': gu_notravel, 
    '여행': gu_traveled
})

fs.to_csv(df, './data/gu_travel.csv', index=True)

In [9]:
# 확진일 일별 분석
gu_daily_discharged = covid19[covid19['상태'] == '퇴원'].groupby(['지역', '날짜']).size()
gu_daily_dead = covid19[covid19['상태'] == '사망'].groupby(['지역', '날짜']).size()
gu_daily_hospitalized = covid19[covid19['상태'].isna()].groupby(['지역', '날짜']).size()

df = pd.DataFrame({
    '퇴원': gu_daily_discharged, 
    '사망': gu_daily_dead, 
    '입원': gu_daily_hospitalized
})

fs.to_csv(df, './data/gu_daily.csv', index=True)

In [10]:
# 확진일 월별 분석
gu_monthly_discharged = covid19[covid19['상태'] == '퇴원'].groupby(['지역', pd.Grouper(key='날짜', freq='1M')]).size()
gu_monthly_dead = covid19[covid19['상태'] == '사망'].groupby(['지역', pd.Grouper(key='날짜', freq='1M')]).size()
gu_monthly_hospitalized = covid19[covid19['상태'].isna()].groupby(['지역', pd.Grouper(key='날짜', freq='1M')]).size()

df = pd.DataFrame({
    '퇴원': gu_monthly_discharged, 
    '사망': gu_monthly_dead, 
    '입원': gu_monthly_hospitalized
})

fs.to_csv(df, './data/gu_monthly.csv', index=True)